In [ ]:
from dialogue2graph.metrics.no_llm_metrics.metrics import (
    match_roles_multi_utterance,
    is_correct_length_multi_utterance
)
import json

from dialogue2graph.pipelines.model_storage import ModelStorage
from dialogue2graph.datasets.augment_dialogues.prompts import augmentation_prompt_3_vars

/home/olshevskaya/miniconda3/envs/conda_env_py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [2]:
from dialogue2graph.pipelines.core.dialogue import DialogueMessage, Dialogue

In [3]:
with open('../data/gen_dataset_augmented_0-402_nodes_edges_v5.json', "r") as fp:
    data = json.load(fp)
example = data[0]
dialogues = example['dialogues']
augmented_dialogues = example['augmented_dialogues']

dia = dialogues[0]
aug_dia = augmented_dialogues[0]

In [4]:
# Initialize model storage
model_storage = ModelStorage()

# Add models to storage
model_storage.add(
    key="generation-llm",
    config={"model_name": "gpt-4o-mini-2024-07-18", "temperature": 0.7},
    model_type="llm"
)

model_storage.add(
    key="formatting-llm", 
    config={"model_name": "gpt-3.5-turbo", "temperature": 0.7},
    model_type="llm"
)

2025-04-11 15:22:20,180 - dialogue2graph.pipelines.model_storage - INFO - Added llm model 'generation-llm' to storage
2025-04-11 15:22:20,248 - dialogue2graph.pipelines.model_storage - INFO - Added llm model 'formatting-llm' to storage


## augmentaion.py

In [11]:
from dialogue2graph.datasets.augment_dialogues.augmentation import DialogueAugmenter as DialogueAugmenter_

In [12]:
augmenter = DialogueAugmenter_(
    model_storage=model_storage,
    generation_llm="generation-llm",
    formatting_llm="formatting-llm"
)
augmenter

DialogueAugmenter(model_storage=ModelStorage(storage={'generation-llm': StoredData(key='generation-llm', config={'model_name': 'gpt-4o-mini-2024-07-18', 'temperature': 0.7}, model_type='llm', model=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7fa63f59cc50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fa63ccdf550>, root_client=<openai.OpenAI object at 0x7fa63f591910>, root_async_client=<openai.AsyncOpenAI object at 0x7fa63ccda290>, model_name='gpt-4o-mini-2024-07-18', model_kwargs={}, openai_api_key=SecretStr('**********'))), 'formatting-llm': StoredData(key='formatting-llm', config={'model_name': 'gpt-3.5-turbo', 'temperature': 0.7}, model_type='llm', model=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7fa63cd090d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fa63cd0ddd0>, root_client=<openai.OpenAI object at

In [ ]:
result = augmenter.invoke(
    dialogue=dia['messages'],
    topic=example['topic'],
    prompt=augmentation_prompt_3_vars
)
result

2025-04-11 13:23:43,333 - httpx - INFO - HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


[{'participant': 'assistant',
  'text': ['Hello! How can I help you today?',
   'Hi! What assistance do you need today?',
   'Hey there! How can I support you today?']},
 {'participant': 'user',
  'text': ['I’d like to make a purchase.',
   'I want to place an order for something.',
   "I'm looking to order a product."]},
 {'participant': 'assistant',
  'text': ['Awesome! Which item are you interested in?',
   'Sounds good! What product do you have in mind?',
   'Fantastic! What would you like to order?']},
 {'participant': 'user',
  'text': ["I'm interested in a t-shirt.",
   'I’d like to order a t-shirt.',
   'I want to get a t-shirt.']},
 {'participant': 'assistant',
  'text': ['What size do you need?',
   'Could you tell me your size?',
   'Which size would you prefer?']},
 {'participant': 'user',
  'text': ['Medium would be great.',
   'I’ll go with medium, please.',
   'I’d like a medium size.']},
 {'participant': 'assistant',
  'text': ['Do you want that in red or blue?',
   'Wo

In [ ]:
aug_dia = {"id": 1, "messages": result}

In [14]:
match_roles_multi_utterance(dia, aug_dia)

True

In [15]:
is_correct_length_multi_utterance(dia, aug_dia)

True

In [13]:
report = await augmenter.evaluate(
    dialogue=dia,
    topic=example['topic'],
    prompt=augmentation_prompt_3_vars
)

2025-04-11 15:24:57,040 - httpx - INFO - HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


TypeError: list indices must be integers or slices, not str

## augmentation_corrected.py

In [5]:
from dialogue2graph.datasets.augment_dialogues.augmentation_corrected import DialogueAugmenter

In [6]:
dialogue = Dialogue.from_list(dia['messages'])

In [7]:
augmenter = DialogueAugmenter(
    model_storage=model_storage,
    generation_llm="generation-llm",
    formatting_llm="formatting-llm"
)

In [8]:
result = augmenter.invoke(
    dialogue=dialogue,
    topic=example['topic'],
    prompt=augmentation_prompt_3_vars
)

2025-04-11 13:37:57,286 - httpx - INFO - HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
result[:3]

[{'participant': 'assistant',
  'text': ['Hello! How can I help you today?',
   'Hi! What can I do for you today?',
   'Hey there! Need assistance with something?']},
 {'participant': 'user',
  'text': ['I’d like to make an order.',
   'I want to place an order for something.',
   'I’m looking to order a product.']},
 {'participant': 'assistant',
  'text': ['Awesome! Which product are you looking to buy?',
   "That's great! What item are you interested in?",
   'Fantastic! What would you like to order?']}]

In [13]:
aug_dia = {"id": 1, "messages": result}
match_roles_multi_utterance(dia, aug_dia), is_correct_length_multi_utterance(dia, aug_dia)

(True, True)

In [8]:
report = await augmenter.evaluate(
    dialogue=dialogue,
    topic=example['topic'],
    prompt=augmentation_prompt_3_vars
)

2025-04-11 15:22:44,503 - httpx - INFO - HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
report

{'match_roles': True, 'correct_length': True}